In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
// Stops auto-scrolling so entire output is visible: see https://stackoverflow.com/a/41646403

In [ ]:
# Default parameter values. They will be overwritten by papermill notebook parameters.
# This cell must carry the tag "parameters" in its metadata.
from pathlib import Path
import pickle
import codecs

innereye_path = Path.cwd().parent.parent.parent
train_metrics_csv = ""
val_metrics_csv = ""
test_metrics_csv = ""
number_best_and_worst_performing = 20
config= ""

In [ ]:
import sys
print(f"Adding to path: {innereye_path}")
if str(innereye_path) not in sys.path:
    sys.path.append(str(innereye_path))

%matplotlib inline
import matplotlib.pyplot as plt

config = pickle.loads(codecs.decode(config.encode(), "base64"))

from InnerEye.ML.reports.notebook_report import print_header
from InnerEye.ML.reports.classification_multilabel_report import print_metrics_for_thresholded_output_for_all_prediction_targets

import warnings
warnings.filterwarnings("ignore")
plt.rcParams['figure.figsize'] = (20, 10)

#convert params to Path
train_metrics_csv = Path(train_metrics_csv)
val_metrics_csv = Path(val_metrics_csv)
test_metrics_csv = Path(test_metrics_csv)

# Train Metrics (for label combinations)

In [ ]:
if train_metrics_csv.is_file():
    print_metrics_for_thresholded_output_for_all_prediction_targets(val_metrics_csv=train_metrics_csv,
                                                                    test_metrics_csv=train_metrics_csv,
                                                                    config=config)

# Validation Metrics (for label combinations)

In [ ]:
if val_metrics_csv.is_file():
    print_metrics_for_thresholded_output_for_all_prediction_targets(val_metrics_csv=val_metrics_csv,
                                                                    test_metrics_csv=val_metrics_csv,
                                                                    config=config)

# Test Metrics (for label combinations)

In [ ]:
if val_metrics_csv.is_file() and test_metrics_csv.is_file():
    print_metrics_for_thresholded_output_for_all_prediction_targets(val_metrics_csv=val_metrics_csv,
                                                                    test_metrics_csv=test_metrics_csv,
                                                                    config=config)